On utilise depuis peu le bloc de colonnes B_... de la table borrowers pour gérer les contentieux.
Ce bloc n'est plus accessible depuis assez longtemps.

Questions :
- on sait que ce bloc contient des informations autres que celles relatives aux contentieux => les identifier, pour éventuellement les supprimer ou les récuperer dans un autre bloc
- une fois que l'adhérent n'est plus en contentieux, les infos relatives au contentieux doivent être supprimées. Cela n'a jamais été fait, mettre au point une méthode d'analyse pour identifier les cartes concernées.

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

from kiblib.utils.db import DbConn

In [2]:
db_conn = DbConn().create_engine()

# Préparation des données
- on part de la table borrowers pour créer un dataframe "adherents"
- création du dataframe "adh_bloc_ctx" : on ne garde du dataframe "adherents" que les colonnes permettant de bien caractériser l'adhérent (âge, type carte, site, date inscription, date expiration) et les colonnes du bloc B_...
- on ajoute à "adh_bloc_ctx" une colonne "contentieux", avec une valeur vrai/faux, via une jointure sur un dataframe reprenant les adhérents en contentieux.

In [3]:
#query = """SELECT * 
#FROM koha_prod.borrowers b"""
#adherents = pd.read_sql(query, db_conn)

adherents = pd.read_json("http://cataloguekoha.ntrbx.local/cgi-bin/koha/rest.pl/user")

In [4]:
len(adherents)

26894

In [5]:
#adherents.columns

In [6]:
adh_bloc_ctx = adherents[['borrowernumber', 'cardnumber', 'userid', 'B_streetnumber',
       'B_streettype', 'B_address', 'B_address2', 'B_city', 'B_state',
       'B_zipcode', 'B_country', 'B_email', 'B_phone', 'contactnote', 'dateofbirth',
       'branchcode', 'categorycode', 'dateenrolled', 'dateexpiry']]
adh_bloc_ctx = adh_bloc_ctx[adh_bloc_ctx['categorycode'].isin(["CSVT", "CSLT", "BIBL", "MEDB", "MEDA", "MEDC", "MEDP", "COLD", "COLI"])]

In [7]:
#adh_bloc_ctx = adh_bloc_ctx.rename(columns={"cardnumber":"numéro de carte",
#                              "B_address":"date de l'appel téléphonique",
#                              "B_address2":"résultat de l'appel",
#                              "B_city":"date demande création tiers",
#                              "B_state":"numéro du titre de recette",
#                              "B_zipcode":"numéro de tiers GF",
#                              "B_country":"date création titre de recette",
#                              "dateofbirth":"date de naissance",
#                              "branchcode":"site de rattachement",
#                              "categorycode":"type de carte",
#                              "dateenrolled":"date d'inscription",
#                              "dateexpiry":"date d'expiration"})

In [8]:
# on crée un dataframe avec les adhérents en contentieux
query = """SELECT DISTINCT(borrowernumber), DATE(date_due) as date_due
FROM koha_prod.issues i
WHERE date_due + INTERVAL 28 DAY <= CURDATE()
ORDER BY date_due"""
contentieux = pd.read_sql(query, db_conn)
contentieux['contentieux'] = True

In [9]:
adh_bloc_ctx = adh_bloc_ctx.merge(contentieux, how='left', on='borrowernumber')
adh_bloc_ctx.loc[adh_bloc_ctx['contentieux'].isna(), 'contentieux'] = False

In [10]:
adh_bloc_ctx['contentieux'].value_counts()

False    24249
True      2241
Name: contentieux, dtype: int64

In [11]:
adh_bloc_ctx

,borrowernumber,cardnumber,userid,B_streetnumber,B_streettype,B_address,B_address2,B_city,B_state,B_zipcode,...,B_email,B_phone,contactnote,dateofbirth,branchcode,categorycode,dateenrolled,dateexpiry,date_due,contentieux
0,9,X0002058641,X0002058641,,,,,,,,...,,,,2002-09-11,MED,BIBL,2007-05-02,2021-12-02,NaN,False
1,15,X0001920261,X0001920261,,,,,,,,...,,,,2009-09-08,MED,BIBL,2013-03-30,2021-09-29,NaN,False
2,25,X0002404233,X0002404233,None,None,,,,,,...,,,,1996-08-07,MED,BIBL,2009-07-17,2023-12-30,NaN,False
3,28,X0001659292,X0001659292,,,,,,,,...,,,,2007-12-21,MED,BIBL,2012-02-08,2022-09-18,NaN,False
4,42,X0002437651,X0002437651,,,,,,,,...,,,,1970-10-23,MED,BIBL,2005-03-31,2021-08-18,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26485,73521,X0002747668,X0002747668,,,,,,,,...,,,,2006-10-07,MED,BIBL,2023-07-18,2024-07-18,NaN,False
26486,73522,X0002747101,X0002747101,,,,,,,,...,,,,1972-03-09,MED,BIBL,2023-07-18,2024-07-18,NaN,False
26487,73523,X0002747095,X0002747095,,,,,,,,...,,,,1997-06-07,MED,CSLT,2023-07-18,2024-07-18,NaN,False
26488,73524,X0002747040,X0002747040,,,,,,,,...,,,,2016-06-08,MED,BIBL,2023-07-18,2024-07-18,NaN,False


In [12]:
adh_bloc_ctx[adh_bloc_ctx['contentieux'] == True]

,borrowernumber,cardnumber,userid,B_streetnumber,B_streettype,B_address,B_address2,B_city,B_state,B_zipcode,...,B_email,B_phone,contactnote,dateofbirth,branchcode,categorycode,dateenrolled,dateexpiry,date_due,contentieux
8,72,X0002315447,X0002315447,,,2021-01-19,usager joint,2021-02-09,20210034,91611,...,,,C0005382767\r\nC2500004195\r\nC2500012698\r\nC...,2012-04-05,MED,BIBL,2019-10-22,2021-06-22,2019-11-12,True
32,237,X0002217536,X0002217536,None,None,,,,,,...,,,Titre émis avant 2021,2000-06-30,MED,BIBL,2011-05-06,2018-07-11,2018-02-24,True
51,344,X0002350554,X0002350554,None,None,,,,,,...,,,Titre émis avant 2021,2007-10-19,MED,MEDC,2009-04-01,2018-10-21,2018-02-24,True
65,478,X0001770153,X0001770153,None,None,,,,,,...,,,,2005-03-12,MED,BIBL,2011-11-22,2020-08-19,2018-02-24,True
106,842,X0001529798,X0001529798,,,2023-07-15,message vocal,,,,...,,,,1969-03-03,MED,BIBL,2006-09-08,2023-10-18,2023-05-30,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25663,72670,X0002654768,X0002654768,,,2023-04-04,message vocal,,,,...,,,,2016-03-31,MED,BIBL,2023-05-06,2024-05-06,2023-06-03,True
25892,72899,X0002610955,X0002610955,,,,,,,,...,,,,2020-03-31,MED,BIBL,2023-05-24,2024-05-24,2023-06-14,True
25953,72962,X0002743431,X0002743431,,,,,,,,...,,,,2006-02-02,MED,BIBL,2023-05-26,2024-05-26,2023-06-16,True
25971,72980,X0002743073,X0002743073,,,,,,,,...,,,,2003-12-25,MED,BIBL,2023-05-27,2024-05-27,2023-06-17,True


In [13]:
# pour chaque champs, on a parfois des valeurs vides : on les passe en 'None' pour faciliter les requêtes
for c in adh_bloc_ctx.columns:
    adh_bloc_ctx.loc[adh_bloc_ctx[c] == '', c] = np.nan

# on analyse les valeurs contenues dans le bloc B_
L'objectif est de n'avoir que des valeurs en lien avec les contentieux.

In [14]:
for c in ['B_streetnumber','B_streettype','B_address','B_address2','B_city',
          'B_state','B_zipcode','B_country','B_email','B_phone', 'contactnote']:
    print(c)
    print(adh_bloc_ctx[c].value_counts())
    print('---------------------------------------')

B_streetnumber
Series([], Name: B_streetnumber, dtype: int64)
---------------------------------------
B_streettype
Series([], Name: B_streettype, dtype: int64)
---------------------------------------
B_address
2023-01-14    95
2022-07-06    93
2021-10-22    86
2021-01-28    78
2021-08-10    75
              ..
2020-10-23     1
2020-08-21     1
2021-08-11     1
2020-09-05     1
2023-06-23     1
Name: B_address, Length: 164, dtype: int64
---------------------------------------
B_address2
message vocal        1582
usager joint          888
téléphone inconnu     721
Name: B_address2, dtype: int64
---------------------------------------
B_city
2023-01-25    26
2021-05-22    26
2021-07-27    26
2021-04-06    25
2022-05-21    25
              ..
2021-01-27     1
2022-07-06     1
2021-07-24     1
2021-02-03     1
2021-01-28     1
Name: B_city, Length: 143, dtype: int64
---------------------------------------
B_state
0           500
20210064      6
20220102      6
20220101      5
20220214      

On constate que 'B_streetnumber','B_streettype','B_email' et 'B_phone' ne contiennent aucune info, on peut donc les supprimer du dataframe.

In [15]:
adh_bloc_ctx = adh_bloc_ctx.drop(columns=['B_streetnumber','B_streettype','B_email', 'B_phone'])

In [16]:
adh_bloc_ctx

,borrowernumber,cardnumber,userid,B_address,B_address2,B_city,B_state,B_zipcode,B_country,contactnote,dateofbirth,branchcode,categorycode,dateenrolled,dateexpiry,date_due,contentieux
0,9.0,X0002058641,X0002058641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002-09-11,MED,BIBL,2007-05-02,2021-12-02,NaN,False
1,15.0,X0001920261,X0001920261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-09-08,MED,BIBL,2013-03-30,2021-09-29,NaN,False
2,25.0,X0002404233,X0002404233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996-08-07,MED,BIBL,2009-07-17,2023-12-30,NaN,False
3,28.0,X0001659292,X0001659292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-12-21,MED,BIBL,2012-02-08,2022-09-18,NaN,False
4,42.0,X0002437651,X0002437651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970-10-23,MED,BIBL,2005-03-31,2021-08-18,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26485,73521.0,X0002747668,X0002747668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-10-07,MED,BIBL,2023-07-18,2024-07-18,NaN,False
26486,73522.0,X0002747101,X0002747101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1972-03-09,MED,BIBL,2023-07-18,2024-07-18,NaN,False
26487,73523.0,X0002747095,X0002747095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-06-07,MED,CSLT,2023-07-18,2024-07-18,NaN,False
26488,73524.0,X0002747040,X0002747040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-06-08,MED,BIBL,2023-07-18,2024-07-18,NaN,False


In [17]:
# ppur plus de facilité, on peut renommer les champs dédiés à la gestion des contentieux
adh_bloc_ctx = adh_bloc_ctx.rename(columns={"B_address":"date_appel_telephonique",
                              "B_address2":"resultat_appel",
                              "B_city":"date_demande_creation_tiers",
                              "B_state":"numero_titre_recette",
                              "B_zipcode":"numero_tiers_GF",
                              "B_country":"date_creation_titre_recette",
                              "contactnote":"documents_a_payer"})

In [18]:
adh_bloc_ctx

,borrowernumber,cardnumber,userid,date_appel_telephonique,resultat_appel,date_demande_creation_tiers,numero_titre_recette,numero_tiers_GF,date_creation_titre_recette,documents_a_payer,dateofbirth,branchcode,categorycode,dateenrolled,dateexpiry,date_due,contentieux
0,9.0,X0002058641,X0002058641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002-09-11,MED,BIBL,2007-05-02,2021-12-02,NaN,False
1,15.0,X0001920261,X0001920261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-09-08,MED,BIBL,2013-03-30,2021-09-29,NaN,False
2,25.0,X0002404233,X0002404233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996-08-07,MED,BIBL,2009-07-17,2023-12-30,NaN,False
3,28.0,X0001659292,X0001659292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-12-21,MED,BIBL,2012-02-08,2022-09-18,NaN,False
4,42.0,X0002437651,X0002437651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970-10-23,MED,BIBL,2005-03-31,2021-08-18,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26485,73521.0,X0002747668,X0002747668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-10-07,MED,BIBL,2023-07-18,2024-07-18,NaN,False
26486,73522.0,X0002747101,X0002747101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1972-03-09,MED,BIBL,2023-07-18,2024-07-18,NaN,False
26487,73523.0,X0002747095,X0002747095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997-06-07,MED,CSLT,2023-07-18,2024-07-18,NaN,False
26488,73524.0,X0002747040,X0002747040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-06-08,MED,BIBL,2023-07-18,2024-07-18,NaN,False


In [19]:
adh_bloc_ctx_with_data = adh_bloc_ctx[(~adh_bloc_ctx['date_appel_telephonique'].isna())
                                    | (~adh_bloc_ctx['resultat_appel'].isna())
                                    | (~adh_bloc_ctx['date_demande_creation_tiers'].isna())
                                    | (~adh_bloc_ctx['numero_titre_recette'].isna())
                                    | (~adh_bloc_ctx['numero_tiers_GF'].isna())
                                    | (~adh_bloc_ctx['date_creation_titre_recette'].isna())
                                    | (~adh_bloc_ctx['documents_a_payer'].isna())]
adh_bloc_ctx_with_data

,borrowernumber,cardnumber,userid,date_appel_telephonique,resultat_appel,date_demande_creation_tiers,numero_titre_recette,numero_tiers_GF,date_creation_titre_recette,documents_a_payer,dateofbirth,branchcode,categorycode,dateenrolled,dateexpiry,date_due,contentieux
8,72.0,X0002315447,X0002315447,2021-01-19,usager joint,2021-02-09,20210034,91611,2021-02-09,C0005382767\r\nC2500004195\r\nC2500012698\r\nC...,2012-04-05,MED,BIBL,2019-10-22,2021-06-22,2019-11-12,True
27,203.0,X0001954258,X0001954258,2021-02-06,message vocal,NaN,NaN,NaN,NaN,NaN,1998-08-04,MED,BIBL,2011-12-10,2023-05-07,NaN,False
30,212.0,X0002003986,X0002003986,2021-10-22,message vocal,NaN,NaN,NaN,NaN,NaN,2002-06-08,MED,BIBL,2013-10-02,2021-10-21,NaN,False
31,215.0,X0002094755,X0002094755,2023-05-17,usager joint,NaN,NaN,NaN,NaN,NaN,1980-08-29,MED,MEDB,2008-06-13,2023-08-26,NaN,False
32,237.0,X0002217536,X0002217536,NaN,NaN,NaN,NaN,NaN,2019-08-27,Titre émis avant 2021,2000-06-30,MED,BIBL,2011-05-06,2018-07-11,2018-02-24,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25663,72670.0,X0002654768,X0002654768,2023-04-04,message vocal,NaN,NaN,NaN,NaN,NaN,2016-03-31,MED,BIBL,2023-05-06,2024-05-06,2023-06-03,True
25728,72735.0,X0002609010,X0002609010,2023-05-03,message vocal,NaN,NaN,NaN,NaN,NaN,1973-05-09,MED,BIBL,2023-05-10,2024-05-10,NaN,False
25907,72914.0,X0002611129,X0002611129,NaN,NaN,NaN,NaN,92421,NaN,NaN,2016-05-26,MED,BIBL,2023-05-24,2024-05-24,NaN,False
25926,72933.0,X0002744513,X0002744513,2022-11-05,message vocal,2023-04-29,0,0,2023-04-29,C2500011914 - 10.95,2014-01-03,MED,BIBL,2023-05-25,2024-05-25,NaN,False


pour chaque colonne contentieux, les valeurs admises sont les suivantes :
- date_appel_telephonique : date au format AAAA-MM-JJ
- resultat_appel : ['usager joint', 'message vocal', 'téléphone inconnu']
- date_demande_creation_tiers : date au format AAAA-MM-JJ
- numero_titre_recette : numéro
- numero_tiers_GF : nombre
- date_creation_titre_recette : date au format AAAA-MM-JJ
- documents_a_payer : texte

In [20]:
# on identifie les lignes qui posent pb
# 
adh_bloc_ctx_with_data_pb = adh_bloc_ctx_with_data[
    ( 
        (~adh_bloc_ctx_with_data['date_appel_telephonique'].astype(str).str.match("\d{4}-\d{2}-\d{2}"))
      & (adh_bloc_ctx_with_data['date_appel_telephonique'].notna()) 
    )
    
      | (  
        (~adh_bloc_ctx_with_data['resultat_appel'].isin(['usager joint', 'message vocal', 'téléphone inconnu']))
      & (adh_bloc_ctx_with_data['resultat_appel'].notna()) 
    )
    
  | (  
        (~adh_bloc_ctx_with_data['numero_titre_recette'].astype(str).str.match("\d"))
      & (adh_bloc_ctx_with_data['numero_titre_recette'].notna()) 
    )
    
  | (  
        (~adh_bloc_ctx_with_data['numero_tiers_GF'].astype(str).str.match("\d"))
      & (adh_bloc_ctx_with_data['numero_tiers_GF'].notna()) 
    )  
    
  | (  
        (~adh_bloc_ctx_with_data['date_demande_creation_tiers'].astype(str).str.match("\d{4}-\d{2}-\d{2}"))
      & (adh_bloc_ctx_with_data['date_demande_creation_tiers'].notna()) 
    )
    
  | (  
        (~adh_bloc_ctx_with_data['date_creation_titre_recette'].astype(str).str.match("\d{4}-\d{2}-\d{2}"))
      & (adh_bloc_ctx_with_data['date_creation_titre_recette'].notna()) 
    )
]
len(adh_bloc_ctx_with_data_pb)

0

In [21]:
adh_bloc_ctx_with_data_pb[adh_bloc_ctx_with_data_pb['contentieux'] ==  False]

,borrowernumber,cardnumber,userid,date_appel_telephonique,resultat_appel,date_demande_creation_tiers,numero_titre_recette,numero_tiers_GF,date_creation_titre_recette,documents_a_payer,dateofbirth,branchcode,categorycode,dateenrolled,dateexpiry,date_due,contentieux


In [22]:
adh_bloc_ctx_with_data_pb.groupby("contentieux").size()

Series([], dtype: int64)

vérifier si pour ces cartes les colonnes contentieux peuvent être mises à Null

# Cartes avec données de contentieux sans contentieux

Explication de la requête ci dessous : 
* ligne 1 : `adh_bloc_ctx_with_data_without_ctx` = tous les documents de `adh_bloc_ctx_with_data` dont le `borrowernumber` ne correspond pas au borrowernumber des documents en contentieux qui ont une erreur de graphie (`adh_bloc_ctx_with_data_pb` )`adh_bloc_ctx_with_data_pb`)

In [23]:
adh_bloc_ctx_with_data_without_ctx = adh_bloc_ctx_with_data[~adh_bloc_ctx_with_data['borrowernumber'].isin(adh_bloc_ctx_with_data_pb['borrowernumber'])]
adh_bloc_ctx_with_data_without_ctx = adh_bloc_ctx_with_data_without_ctx[adh_bloc_ctx_with_data_without_ctx['contentieux'] == False]
adh_bloc_ctx_with_data_without_ctx #.to_excel("")

,borrowernumber,cardnumber,userid,date_appel_telephonique,resultat_appel,date_demande_creation_tiers,numero_titre_recette,numero_tiers_GF,date_creation_titre_recette,documents_a_payer,dateofbirth,branchcode,categorycode,dateenrolled,dateexpiry,date_due,contentieux
27,203.0,X0001954258,X0001954258,2021-02-06,message vocal,NaN,NaN,NaN,NaN,NaN,1998-08-04,MED,BIBL,2011-12-10,2023-05-07,NaN,False
30,212.0,X0002003986,X0002003986,2021-10-22,message vocal,NaN,NaN,NaN,NaN,NaN,2002-06-08,MED,BIBL,2013-10-02,2021-10-21,NaN,False
31,215.0,X0002094755,X0002094755,2023-05-17,usager joint,NaN,NaN,NaN,NaN,NaN,1980-08-29,MED,MEDB,2008-06-13,2023-08-26,NaN,False
40,287.0,X0002272290,X0002272290,2023-05-17,message vocal,NaN,NaN,NaN,NaN,NaN,2001-11-12,MED,BIBL,2014-02-15,2024-02-21,NaN,False
44,299.0,X0002564296,X0002564296,2021-02-06,usager joint,2021-03-09,20210111,50028,2021-03-10,Titre de recette payé à la Trésorerie,2010-08-22,MED,BIBL,2012-07-07,2023-06-24,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25651,72658.0,X0002654935,X0002654935,2020-09-09,message vocal,2022-04-16,0,59121,2022-04-16,C2500012203 - 11.70,2017-08-11,MED,BIBL,2023-05-06,2024-05-06,NaN,False
25728,72735.0,X0002609010,X0002609010,2023-05-03,message vocal,NaN,NaN,NaN,NaN,NaN,1973-05-09,MED,BIBL,2023-05-10,2024-05-10,NaN,False
25907,72914.0,X0002611129,X0002611129,NaN,NaN,NaN,NaN,92421,NaN,NaN,2016-05-26,MED,BIBL,2023-05-24,2024-05-24,NaN,False
25926,72933.0,X0002744513,X0002744513,2022-11-05,message vocal,2023-04-29,0,0,2023-04-29,C2500011914 - 10.95,2014-01-03,MED,BIBL,2023-05-25,2024-05-25,NaN,False


a priori, données de contentieux à supprimer : attention à possibilité de faire des stats

In [24]:
import requests
import json

def mod_borrower(userid, data):
    api_url = "http://cataloguekoha.ntrbx.local/cgi-bin/koha/rest.pl/user"
    url = f"{api_url}/{userid}"
    data = json.dumps(data)
    data2mod = f"data={data}"
    print(data2mod)
    response = requests.put(url, data=data2mod)
    print(f"{userid} : {response.content}")

In [25]:
data = {"B_address": None,
        "B_address2": None,
        "B_city": None,
        "B_state": None,
        "B_zipcode": None,
        "B_country": None,
        "contactnote": None}
userid = 'X0002525563'
mod_borrower(userid, data)

data={"B_address": null, "B_address2": null, "B_city": null, "B_state": null, "B_zipcode": null, "B_country": null, "contactnote": null}
X0002525563 : b'{\n   "modified_fields" : {},\n   "success" : true\n}\n'
